In [6]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue May 28 16:09:13 2019

@author: admin
"""

import os
import numpy as np
import tensorflow as tf
import h5py
import math
#import data_preprocessing as dp


import matplotlib.pyplot as plt
from urllib.request import urlopen,urlretrieve
from PIL import Image
from tqdm import tqdm_notebook
from sklearn.utils import shuffle
import cv2
#from resnet_utils import *

from keras.models import load_model
from sklearn.datasets import load_files   
from keras.utils import np_utils
from glob import glob
from keras import applications
from keras.preprocessing.image import ImageDataGenerator 
from keras import optimizers
from keras.models import Sequential,Model,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.callbacks import TensorBoard,ReduceLROnPlateau,ModelCheckpoint


################# prepare data #################
'''
load dataset
normalize
dummy variables (encode y)
constants

'''

################ model training #################
'''
build up architecture


'''

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue May 28 17:12:59 2019

@author: admin
"""

import os
import numpy as np
import tensorflow as tf
import h5py
import math


'''
load dataset from folder
'''
def load_dataset():
    train_dataset = h5py.File('./training-data-0528.h5', "r")
    train_set_x_orig = np.array(train_dataset['X'][:])
    train_set_y_orig = np.array(train_dataset["y"][:])

    test_dataset = h5py.File('./testing-data-0528.h5', "r")
    test_set_x_orig = np.array(test_dataset['X'][:])
    test_set_y_orig = np.array(test_dataset["y"][:])
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))

    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig


def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
    """
    Creates a list of random minibatches from (X, Y)
    
    Arguments:
    X -- input data, of shape (input size, number of examples) (m, Hi, Wi, Ci)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat), of shape (1, number of examples) (m, n_y)
    mini_batch_size - size of the mini-batches, integer
    seed -- this is only for the purpose of grading, so that you're "random minibatches are the same as ours.
    
    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """
    
    m = X.shape[0]                  # number of training examples
    mini_batches = []
    np.random.seed(seed)
    
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[permutation,:,:,:]
    shuffled_Y = Y[permutation,:]

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:,:,:]
        mini_batch_Y = shuffled_Y[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[num_complete_minibatches * mini_batch_size : m,:,:,:]
        mini_batch_Y = shuffled_Y[num_complete_minibatches * mini_batch_size : m,:]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches


def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y


def forward_propagation_for_predict(X, parameters):
    """
    Implements the forward propagation for the model: LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SOFTMAX
    
    Arguments:
    X -- input dataset placeholder, of shape (input size, number of examples)
    parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3"
                  the shapes are given in initialize_parameters
    Returns:
    Z3 -- the output of the last LINEAR unit
    """
    
    # Retrieve the parameters from the dictionary "parameters" 
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3'] 
                                                           # Numpy Equivalents:
    Z1 = tf.add(tf.matmul(W1, X), b1)                      # Z1 = np.dot(W1, X) + b1
    A1 = tf.nn.relu(Z1)                                    # A1 = relu(Z1)
    Z2 = tf.add(tf.matmul(W2, A1), b2)                     # Z2 = np.dot(W2, a1) + b2
    A2 = tf.nn.relu(Z2)                                    # A2 = relu(Z2)
    Z3 = tf.add(tf.matmul(W3, A2), b3)                     # Z3 = np.dot(W3,Z2) + b3
    
    return Z3

def predict(X, parameters):
    
    W1 = tf.convert_to_tensor(parameters["W1"])
    b1 = tf.convert_to_tensor(parameters["b1"])
    W2 = tf.convert_to_tensor(parameters["W2"])
    b2 = tf.convert_to_tensor(parameters["b2"])
    W3 = tf.convert_to_tensor(parameters["W3"])
    b3 = tf.convert_to_tensor(parameters["b3"])
    
    params = {"W1": W1,
              "b1": b1,
              "W2": W2,
              "b2": b2,
              "W3": W3,
              "b3": b3}
    
    x = tf.placeholder("float", [12288, 1])
    
    z3 = forward_propagation_for_predict(x, params)
    p = tf.argmax(z3)
    
    sess = tf.Session()
    prediction = sess.run(p, feed_dict = {x: X})
        
    return prediction



'''
normalize the image data
'''
def normalize():
    # Normalize image vectors
    X_train = X_train_orig/255.
    X_test = X_test_orig/255.
    return X_train, X_test
    
'''
one hot encoding y labels
'''
def encode_y():
    Y_train = convert_to_one_hot(Y_train_orig, 3).T
    Y_test = convert_to_one_hot(Y_test_orig, 3).T
    return Y_train, Y_test
    
def check_shape():
    print ("number of training examples = " + str(X_train.shape[0]))
    print ("number of test examples = " + str(X_test.shape[0]))
    print ("X_train shape: " + str(X_train.shape))
    print ("Y_train shape: " + str(Y_train.shape))
    print ("X_test shape: " + str(X_test.shape))
    print ("Y_test shape: " + str(Y_test.shape))
    
    
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig = load_dataset()
normalize()

X_train, X_test = normalize()
Y_train, Y_test = encode_y()

img_height,img_width = 64,64 
num_classes = 3

base_model = applications.resnet50.ResNet50(weights= None, include_top=False, input_shape= (img_height,img_width,3))    

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(num_classes, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)    
    
from keras.optimizers import SGD, Adam
# sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
adam = Adam(lr=0.0001)
model.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs = 100, batch_size = 64)





/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Epoch 1/100
2023/2023 [==============================] - 18s 9ms/step - loss: 1.9969 - acc: 0.4498
Epoch 2/100
2023/2023 [==============================] - 4s 2ms/step - loss: 1.5568 - acc: 0.5586
Epoch 3/100
2023/2023 [==============================] - 4s 2ms/step - loss: 1.3128 - acc: 0.6041
Epoch 4/100
2023/2023 [==============================] - 4s 2ms/step - loss: 1.1953 - acc: 0.6490
Epoch 5/100
2023/2023 [==============================] - 4s 2ms/step - loss: 1.0772 - acc: 0.6797
Epoch 6/100
2023/2023 [==============================] - 4s 2ms/step - loss: 0.9001 - acc: 0.7360
Epoch 7/100
2023/2023 [==============================] - 4s 2ms/step - loss: 0.7847 - acc: 0.7652
Epoch 8/100
2023/2023 [==============================] - 4s 2ms/step - loss: 0.6079 - acc: 0.7978
Epoch 9/100
2023/2023 [==============================] - 4s 2ms/step - loss: 0.5288 - acc: 0.8389
Epoch 10/100
2023/2023 [==============================] - 4s 2ms/step - loss: 0.5171 - acc: 0.8492
Epoch 11/100
2023/

In [7]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

504/504 [==============================] - 3s 7ms/step
Loss = 1.5083569051727417
Test Accuracy = 0.7698412688951644


In [8]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 70, 70, 3)    0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 32, 32, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 32, 32, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation